# Pandora Papers

## Pandora Papers - The Data Connecting Politicians, Criminals and a Rogue Industry that hides their cash
Original walkthrough can be accessed by running `:play https://guides.neo4j.com/sandbox/icij-offshoreleaks/pandora-papers-examples.html` in Neo4j Browser. Unfortunately, the dataset for Neo4j 5 somehow doesn't contain Pandora Papers data. Thus, if you run it on Neo4j sandbox, you won't get any result.

This walkthrough uses the data for Neo4j 4.4 running on a Docker container. You can execute the script [run_pandora.sh](./run_pandora.sh).



### Discover the relationships between two entity officers who share common entity or adddress

For example, let's say we want to discover the shortest paths between two entity officers through a set of Entity or Address nodes. This is quite easy with Cypher, Neo4j's graph query language.

```sql
MATCH (a:Officer),(b:Officer)
          WHERE a.name CONTAINS 'MATUWANGA' AND a.sourceID STARTS WITH "Pandora Papers"
            AND b.name CONTAINS 'UZUN' AND b.sourceID STARTS WITH "Pandora Papers"
          MATCH p=allShortestPaths((a)-[:officer_of|intermediary_of|registered_address*..10]-(b))
          RETURN p
          LIMIT 50
```
![](img/pan001.png)


### Entity registered by an intermediary

```sql
MATCH (i:Intermediary)-[r:intermediary_of]->(e:Entity)
WHERE i.name CONTAINS "SFM CORPORATE SERVICES S.A." AND e.sourceID STARTS WITH "Pandora Papers"
RETURN i, r, e LIMIT 100
```

![](img/pan002.png)

### Search for Officer nodes by name

```sql
:param officer=>"ORLOV"

MATCH (o:Officer)
WHERE o.name CONTAINS $officer AND o.sourceID STARTS WITH "Pandora Papers"
RETURN o
LIMIT 100
```

### Search for an Officer and find the connections

#### 1st degree

```sql
MATCH (o:Officer)
WHERE o.name CONTAINS $officer AND o.sourceID STARTS WITH "Pandora Papers"
MATCH path = (o)-[r]->(:Entity)
RETURN path LIMIT 100
```

#### 2nd degree

```sql
MATCH (o:Officer) 
WHERE o.name CONTAINS $officer AND o.sourceID STARTS WITH "Pandora Papers"
MATCH path = (o)-[]->(:Entity)
      <-[]-(:Officer)-[]->(:Entity)
RETURN path LIMIT 100
```
![](img/pan003.png)

### Find who is behind an Entity and the roles that they play

```sql
:param entity=>"DANSUNN FINANCE LTD."

MATCH (e:Entity)-[r]-(o:Officer)
WHERE e.name CONTAINS $entity AND e.sourceID STARTS WITH "Pandora Papers"
RETURN *
LIMIT 100
```

![](img/pan004.png)

## Pandora Papers - "The Landlord" Investigation

Original article: https://neo4j.com/developer-blog/digging-into-the-icij-pandora-papers-dataset-with-neo4j/

"The Landlord" investigation article: https://www.icij.org/investigations/pandora-papers/the-land-lords/


In “South Africa’s smart city” Ruslan Goryukhin – a key aide to some of Putin’s closet friends – is reported to be connected to companies involved in the development of the first “post apartheid smart city” – Cradle City in South Africa.

Let’s see what we find in our data, first querying for Goryukhin.

```sql
MATCH path = (o:Officer)-[r]->(:Entity)
WHERE o.name CONTAINS 'GORYUKHIN' AND o.sourceID STARTS WITH "Pandora Papers"
RETURN path LIMIT 100 
```

![](img/pan005.png)

Another person mentioned in the story was “Preston Hampton Haskell IV, the son of a Texas construction billionaire.” Let’s see if we can find him too, and his connections to Goryukhin.

```sql
MATCH (o:Officer), (o2:Officer)
WHERE o.name CONTAINS 'GORYUKHIN' AND o.sourceID STARTS WITH "Pandora Papers" AND o2.name contains 'PRESTON HAMPTON HASKELL'
MATCH path=allShortestPaths((o)-[*]-(o2))
RETURN path LIMIT 25
```
![](img/pan006.png)

The report speaks of a shell company named Kelburn One and Amari Land International Ltd – later renamed to Forum Properties Africa – which unfortunately are not in the published dataset.